<ul class="breadcrumb">
  <li><a href="1_Conventional_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Conventional)</a></li>
  <li><a href="2_Linear_Additive_Genetic_Model.ipynb">Bayesian Linear Additive Genetic Model</a></li> 
  <li><a href="3_Genomic_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Genomic Data)</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Bayesian Linear Mixed Models (Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 1: Load Packages</button> 

In [1]:
include("../../src/JWAS.jl")

JWAS

In [2]:
using JWAS,JWAS.Datasets,DataFrames,CSV

<button type="button" class="btn btn-lg btn-primary">Step 2: Read data</button> 

In [3]:
phenofile  = Datasets.dataset("example","phenotypes.txt")
pedfile    = Datasets.dataset("example","pedigree.txt")
genofile   = Datasets.dataset("example","genotypes.txt")

phenotypes = CSV.read(phenofile,delim = ',',header=true)
pedigree   = get_pedigree(pedfile,separator=",",header=true);

coding pedigree...  10%|███                             |  ETA: 0:00:02

Finished!


coding pedigree... 100%|████████████████████████████████| Time: 0:00:00


In [4]:
head(phenotypes)

,ID,y1,y2,y3,x1,x2,x3,dam
1,a1,-0.06,3.58,-1.18,0.9,2,m,0
2,a2,-0.6,4.9,0.88,0.3,1,f,0
3,a3,-2.07,3.19,0.73,0.7,2,f,0
4,a4,-2.63,6.97,-0.83,0.6,1,m,a2
5,a5,2.31,3.5,-1.52,0.4,2,m,a2
6,a6,0.93,4.87,-0.01,5.0,2,f,a3


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Mixed Model (Genomic data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [5]:
model_equation2 ="y1 = intercept + x1 + x3 + ID + dam
                  y2 = intercept + x1 + x2 + x3 + ID
                  y3 = intercept + x1 + x1*x3 + x2 + ID";

In [6]:
R      = eye(3)
model2 = build_model(model_equation2,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [7]:
set_covariate(model2,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Build Random or Fixed Effects</button> 

In [8]:
G1 = eye(2)
G2 = eye(4)
set_random(model2,"x2",G1);
set_random(model2,"ID dam",pedigree,G2);

INFO: x2 is not found in model equation 1.
INFO: dam is not found in model equation 2.
INFO: dam is not found in model equation 3.


In [9]:
outputMCMCsamples(model2,"x2")

<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [10]:
G3 = eye(3)
add_genotypes(model2,genofile,G3,separator=',');

5 markers on 10 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [11]:
out2=runMCMC(model2,phenotypes,methods="BayesB",estimatePi=true,chain_length=5000,output_samples_frequency=100);

The prior for marker effects covariance matrix were calculated from genetic covariance matrix and π.
Marker effects covariance matrix is 

 0.462963  0.0       0.0     
 0.0       0.462963  0.0     
 0.0       0.0       0.462963


MCMC Information:

methods                                      BayesB
chain_length                                   5000
burnin                                            0
starting_value                                false
printout_frequency                             5001
output_samples_frequency                        100
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                              Dict([1.0, 0.0, 1.0]=>0.0,[0.0, 0.0, 1.0]=>0.0,[0.0, 1.0, 1.0]=>0.0,[1.0, 1.0, 0.0]=>0.0,[0.0, 0.0, 0.0]=>0.0,[0.0, 1.0, 0.0]=>0.0,[1.0, 0.0, 0.0]=>0.0,[1.0, 1.0, 1.0]=>1.0)
estimatePi                           

running MCMC for BayesB...100%|█████████████████████████| Time: 0:00:05


In [12]:
keys(out2)

Base.KeyIterator for a Dict{Any,Any} with 7 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "MCMC samples for: 3:x2"
  "Posterior mean of residual covariance matrix"
  "Posterior mean of marker effects"
  "MCMC samples for: 2:x2"
  "Posterior mean of location parameters"
  "Posterior mean of Pi"

In [13]:
out2["MCMC samples for: 2:x2"]

51×2 Array{Any,2}:
   "2:x2 : 2"    "2:x2 : 1"
  0.298073     -0.125442   
  0.214529     -0.0506464  
  0.725078      1.30093    
 -0.0656984    -0.202652   
  0.732701      0.759989   
 -0.0134146     0.427325   
 -1.86321      -0.691468   
  0.409735      0.168969   
 -0.897083     -1.46067    
  0.752416     -0.12261    
  0.0804155    -0.296188   
 -0.551792     -0.0389822  
  ⋮                        
  0.974803      0.649589   
  0.48591      -0.167352   
  0.239765      0.756294   
  0.419287      0.311845   
 -0.609337     -0.615088   
  0.797771     -0.290773   
  0.364634      0.577609   
 -0.0598853    -0.575038   
 -0.146064      0.00370179 
  2.0686        1.99284    
 -0.805738     -0.435436   
  0.102323     -0.0157767  

<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [14]:
using JWAS:misc

In [15]:
get_breeding_values(model2,"MCMC_samples_y1_marker_effects.txt",
    "MCMC_samples_y2_marker_effects.txt",
    "MCMC_samples_y3_marker_effects.txt")

3-element Array{Any,1}:
 10×3 DataFrames.DataFrame
│ Row │ ID  │ EBV       │ PEV      │
├─────┼─────┼───────────┼──────────┤
│ 1   │ a1  │ 0.315747  │ 0.218987 │
│ 2   │ a2  │ 0.432276  │ 0.353176 │
│ 3   │ a3  │ -0.725705 │ 0.644546 │
│ 4   │ a4  │ -1.37458  │ 0.393229 │
│ 5   │ a5  │ 2.1226    │ 1.01073  │
│ 6   │ a6  │ -1.51249  │ 0.650552 │
│ 7   │ a7  │ 1.10253   │ 0.814434 │
│ 8   │ a8  │ -1.61938  │ 0.572221 │
│ 9   │ a9  │ -1.26769  │ 0.438987 │
│ 10  │ a10 │ 2.52668   │ 1.50179  │
 10×3 DataFrames.DataFrame
│ Row │ ID  │ EBV       │ PEV      │
├─────┼─────┼───────────┼──────────┤
│ 1   │ a1  │ -0.294198 │ 0.191955 │
│ 2   │ a2  │ 0.201048  │ 0.444437 │
│ 3   │ a3  │ 0.106602  │ 0.601136 │
│ 4   │ a4  │ 1.10953   │ 0.460329 │
│ 5   │ a5  │ -1.20268  │ 0.981522 │
│ 6   │ a6  │ 1.21035   │ 0.489878 │
│ 7   │ a7  │ -1.39794  │ 0.827616 │
│ 8   │ a8  │ 1.20865   │ 0.579579 │
│ 9   │ a9  │ 1.11123   │ 0.406031 │
│ 10  │ a10 │ -2.05259  │ 1.24822  │
 10×3 DataFrames.DataFrame
│ Row │